# <center> WizardCoder-15B-V1.0
# <center> 8 bit

#### Resources checkout

In [ ]:
!nvidia-smi

#### Donwnloads (run once)

In [ ]:
!pip3 install  -Uq transformers
!pip3 install  -Uq torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip3 install  -Uq sentencepiece
!pip3 install  -Uq accelerate
!pip3 install  -Uq bitsandbytes

#### Imports

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig
import torch

#### Loading model to VRAM

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_skip_modules = [
        "lm_head", 
        "transformer.wte", 
        "transformer.wpe", 
        "transformer.drop", 
        "transformer.h.0", 
        "transformer.h.1", 
        "transformer.h.39", 
        "transformer.h.38", 
        "transformer.h.37", 
        "transformer.h.36", 
        "transformer.h.35"
        ]
)

BASE_MODEL_ID = 'WizardLM/WizardCoder-15B-V1.0'

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID,
                                            device_map="auto", # {"":0} - load all wieghts to vram
                                            trust_remote_code=True,
                                            quantization_config=quantization_config,
                                            )

#### Prompt

In [ ]:
# Here is your prompt
prompt = '''Write Java class Teacher using Loombok annotation with fields id, name, surname, department.  Write entity for DB TeacherDTO class based on Teacher class. Use JPA jakarta annotations only for TeacherDTO classes to save in data base'''

# Here is template that model can understand your prompt and make response 
prompt_template = f'''Below is an instruction that describes a task. Write a response that appropriately completes the request

### Instruction: {prompt}

### Response:'''

#### Inference

In [ ]:
generation_config = GenerationConfig(
    top_p=0.0,
    top_k=1,
    temperature=0.0,
    max_new_tokens=600,
    pad_token_id = tokenizer.eos_token_id,
    eos_token_id = tokenizer.eos_token_id
)   

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoding = tokenizer(prompt_template, return_tensors="pt").to(device)

In [ ]:
with torch.inference_mode(): 
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache = True
    )

In [ ]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))